In [ ]:
import utils

import glob
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from patchify import patchify, unpatchify
from tensorflow.keras.models import load_model
from ipywidgets import interact


Select the model you wish to check results

In [ ]:
trained_models = "./trained_models"

selected_folder = ""

@interact
def show_files(files=next(os.walk("./trained_models"))[1]):
    global selected_folder
    selected_folder = files
    print("Selected folder:", selected_folder)

Fetch all the data from the folder that includes:

    - Neural Network model
    - The training history
    - Confussion Matrix
    - The classification Report

In [ ]:
model = load_model(os.path.join(trained_models, selected_folder,"unet.h5"), compile=False) # Compile = False as we are only going to predict
classification_report = pd.DataFrame(pd.read_pickle(os.path.join(trained_models, selected_folder, "clf_results.pkl")))
history = pd.read_pickle(os.path.join(trained_models, selected_folder, "history.pkl"))
cf_matrix = np.load(os.path.join(trained_models, selected_folder, "conf_matrix.npy"))

## Learning Curves

In [ ]:
lr_history = np.array(history["lr"])
lr_rate_change_epochs = []
initial_value = lr_history[0]
for i, value in enumerate(np.array(history["lr"])):
    if value != initial_value:
        lr_rate_change_epochs.append(i)
        initial_value = value


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))

for x in lr_rate_change_epochs:
        ax1.axvline(x=x, linestyle="--", color="gray")
ax1.plot(history["loss"], label="training loss")
ax1.plot(history["val_loss"], label="validation loss")
ax1.legend(loc="upper right")

for x in lr_rate_change_epochs:
        ax2.axvline(x=x, linestyle="--", color="gray")
ax2.plot(history["accuracy"], label="training accuracy")
ax2.plot(history["val_accuracy"], label="validation accuracy")
ax2.legend()

plt.show()

## Confussion Matrix

In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'cf_matrix', 'history', and other variables are defined earlier

mossaics_paths = glob.glob("./data/mossaics/training_mossaics/mask*.png")

filled = []
for path in mossaics_paths:
    img = cv2.imread(path, 0)
    size = img.size
    pixels_filled = np.sum(np.where(img != 0, 1, 0))
    fill = np.round(pixels_filled / size, 2)
    filled.append(fill)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

cf_matrix_copy = np.copy(cf_matrix)
cf_matrix_copy = np.round(cf_matrix_copy.astype('float') / cf_matrix_copy.sum(axis=1)[:, np.newaxis],2)
np.fill_diagonal(cf_matrix_copy, 0)
# Plot heatmap with mask and no color bar
sns.heatmap(cf_matrix_copy, annot=True, fmt="g", ax=ax1, cbar=False)  # annot=True to annotate cells, fmt="g" to disable scientific notation

ax1.set_xlabel("Predicted labels")
ax1.set_ylabel("True labels")

ax1.xaxis.set_ticklabels(["Background", "Epithelium", "Blood Vessels", "Stroma", "Adipocytes"], rotation=25)
ax1.yaxis.set_ticklabels(["Background", "Epithelium", "Blood Vessels", "Stroma", "Adipocytes"], rotation=-45)

ax2.plot(history["loss"], label="Training Loss")
ax2.plot(history["val_loss"], label="Validation Loss")

min_loss = min(history["val_loss"])
min_loss_idx = history["val_loss"].index(min_loss)

# Add vertical line
ax2.axvline(x=min_loss_idx, color='red', linestyle='--', linewidth=1)

# Add gray overlay after the vertical line
ax2.axvspan(min_loss_idx, len(history["loss"]), color='gray', alpha=0.3)

# Add text "Early stopping" near the vertical line
ax2.text(min_loss_idx + 1, max(history["val_loss"]), 'Trigger Early \nStopping', color='black', fontsize=10, verticalalignment='top')

# Set x-axis limits between 0 and 80
ax2.set_xlim(-1, len(history["loss"]))

ax2.legend()
plt.show()


## Classification Report

In [ ]:
print("The model accuracy is: " , np.round(classification_report.loc["precision", "accuracy"]*100,2))

In [ ]:
report = classification_report.copy()
report.rename(columns={"0":"Background", "1":'epithelium', "2":'blood_vessels', "3":'stroma', "4":'adipocytes'}, inplace=True)
report.loc["precision"] = np.round(report.loc["precision"]*100,2)
report.loc["recall"] = np.round(report.loc["recall"]*100,2)
report.loc["f1-score"] = np.round(report.loc["f1-score"]*100,2)
report.drop(columns=["accuracy"], inplace=True)
report.loc["support"] = report.loc["support"].astype('int64')
report.head()

## Inspect Training Images

In [ ]:
predictions_path = "./data/predictions"
validation_path = "./data/final_images/validation"
test_path = "./data/final_images/test"

In [ ]:
def check_mossaic_predictions(img_path):
    """
    Given a path get its images and masks and show its predictions vs Ground Truths
    """

    imgs = glob.glob(os.path.join(img_path, "images/*.png"))
    masks = glob.glob(os.path.join(img_path, "masks/*.png"))

    colors = ["black", "green", "red", "blue", "yellow"]
    imin=0
    imax=len(colors)
    cmap = plt.cm.colors.ListedColormap(colors)

    for i in range(len(imgs)):
        img = cv2.imread(imgs[i])/255.0
        mask = cv2.imread(masks[i],0)
        prediction = np.argmax(model.predict(np.expand_dims(img, axis=0), verbose=0), axis=-1)[0]
        fig, [ax1, ax2, ax3, ax4] = plt.subplots(1,4, figsize=(20,20))

        ax1.set_title("Original Image")
        ax1.imshow(img)
        ax2.set_title("Ground Truth Labels")
        ax2.imshow(mask, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest")
        ax3.set_title("Predicted Labels")
        ax3.imshow(prediction, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest")
        ax4.set_title("Pixels on error")
        ax4.imshow((np.where(prediction == mask, 0, 1)))

        ax1.axis("off")
        ax2.axis("off")
        ax3.axis("off")
        ax4.axis("off")

        plt.show()

In [ ]:
check_mossaic_predictions(validation_path)

In [ ]:
check_mossaic_predictions(test_path)

## Inspect real images

Plot Large image for visualization

In [ ]:
cleaned_prediction_path="./data/predictions_cleaned"
colors = ["black", "green", "red", "blue", "yellow"]
imin=0
imax=len(colors)
cmap = plt.cm.colors.ListedColormap(colors)
img_path = glob.glob(os.path.join(cleaned_prediction_path, "*.png"))[0]
img = cv2.imread(img_path,0)
fig, ax1 = plt.subplots(1,1,figsize=(30,30))
ax1.imshow(img, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest")
ax1.axis("off")
plt.show()

In [ ]:
def plot_prediction_stack(prediction_path="./data/predictions" , cleaned_prediction_path="./data/predictions_cleaned"):
    colors = ["black", "green", "red", "blue", "yellow"]
    imin=0
    imax=len(colors)
    cmap = plt.cm.colors.ListedColormap(colors)
    
    predictions = glob.glob(os.path.join(prediction_path, "*.png"))
    predictions_cleaned = glob.glob(os.path.join(cleaned_prediction_path, "*.png"))

    for i in range(len(predictions)):
        fig, [ax1, ax2, ax3] = plt.subplots(1,3, figsize=(10,10))
        prediction_img = cv2.imread(predictions[i], 0)
        prediction_cleaned_img = cv2.imread(predictions_cleaned[i], 0)
        ax1.set_title("Original Prediction")
        ax1.imshow(prediction_img, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest" )
        ax2.set_title("Cleaned Prediction")
        ax2.imshow(prediction_cleaned_img, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest")
        ax3.set_title("Cleaning Impact")
        ax3.imshow(np.where(prediction_img != prediction_cleaned_img, 1, 0))
        ax1.axis("off")
        ax2.axis("off")
        ax3.axis("off")
        plt.show()


In [ ]:
plot_prediction_stack()